In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sbibm.visualisation import fig_metric
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as st
import numpy as np
import seaborn as sns
from typing import Any, Dict, List, Optional

from paper.utils import fig_metric_mpl, extract_diagnostic, fig_diagnostic

In [3]:
from benchmark.utils import compile_df

In [4]:
# df = compile_df("../../../benchmark/results/ess_results")
# df.to_pickle("../../results/benchmark_fig/results_ess.pkl")

In [5]:
df = pd.read_pickle("../../results/benchmark_fig/results_ess.pkl")

In [6]:
df

,task,num_simulations,num_observation,algorithm,seed,acceptance rate,gt in support,iw var,C2ST,MMD,MEDDIST,NLTP,RT,num_simulations_simulator,path,folder
0,bernoulli_glm,10000,1,tsnpe,378249481,"[1.0, -0.7061956524848938, -1.1325324773788452...","[1.0, 0.9120000004768372, 0.9581999778747559, ...","[0.0, 8.749953269958496, 8.064751625061035, 7....",0.68360,0.101535,9.952236,NaN,9.441970,10000.0,/home/michael/Documents/tsnpe_collection/paper...,1
1,two_moons,1000,1,tsnpe,2507482709,"[1.0, -0.1494150459766388, -0.1426071971654892...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 29.17135238647461, 31.944175720214844, 1...",0.56990,0.000144,0.087786,NaN,2.653403,1000.0,/home/michael/Documents/tsnpe_collection/paper...,20
2,bernoulli_glm,1000,4,tsnpe,3544816019,"[1.0, -0.011129045858979225, -0.06083031743764...","[1.0, 0.9891999959945679, 0.9254000186920166, ...","[0.0, 10.409664154052734, 8.683056831359863, 8...",0.92095,0.422845,13.043188,NaN,2.502240,1000.0,/home/michael/Documents/tsnpe_collection/paper...,6
3,two_moons,1000,6,tsnpe,4124588501,"[1.0, -0.18648572266101837, -0.207398220896720...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 40.48292541503906, 31.128049850463867, 2...",0.59650,0.009703,0.095204,NaN,2.272902,1000.0,/home/michael/Documents/tsnpe_collection/paper...,30
4,two_moons,10000,6,tsnpe,2727642344,"[1.0, -0.5939710736274719, -0.7088532447814941...","[1.0, 1.0, 0.9998999834060669, 0.9998000264167...","[0.0, 26.946043014526367, 23.073383331298828, ...",0.52110,0.001586,0.087553,NaN,7.496879,10000.0,/home/michael/Documents/tsnpe_collection/paper...,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,lotka_volterra,100000,8,tsnpe,3400391787,"[1.0, -0.5456125140190125, -0.702458381652832,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 5.178565502166748, 6.475839138031006, 5....",0.92545,0.086961,18.565796,NaN,94.045662,100000.0,/home/michael/Documents/tsnpe_collection/paper...,7
176,lotka_volterra,100000,10,tsnpe,2882890841,"[1.0, -0.2270185649394989, -0.400662899017334,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 10.195143699645996, 7.910343647003174, 7...",0.74355,0.095324,21.170006,NaN,107.273503,100000.0,/home/michael/Documents/tsnpe_collection/paper...,9
177,lotka_volterra,100000,6,tsnpe,270608972,"[1.0, -0.4356890916824341, -1.015923023223877,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 4.384361743927002, 4.9964799880981445, 5...",0.91165,0.491253,25.448696,NaN,109.909546,100000.0,/home/michael/Documents/tsnpe_collection/paper...,5
178,lotka_volterra,100000,3,tsnpe,4198527553,"[1.0, -0.8413360118865967, -1.0629838705062866...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 7.860537052154541, 5.876458168029785, 4....",0.75765,0.074950,16.052723,NaN,110.265633,100000.0,/home/michael/Documents/tsnpe_collection/paper...,2


In [7]:
def extract_diagnostic(df, key="gt in support"):
    full = []
    for n in [1000, 10000, 100000]:
        a = df.query(f"num_simulations=={n}")[key]
        b = np.asarray(a.to_numpy())
        all_lists = []
        for aa in b:
            aaa = aa[1:-2]
            splitted = list(aaa.split(", "))
            new_ = []
            for s in splitted:
                new_.append(float(s))
            all_lists.append(new_)
        all_lists = np.asarray(all_lists)
        full.append(all_lists)
    return np.asarray(full)

In [8]:
# slcp_df = df[df["task"]=='slcp']
# two_moons_df = df[df["task"]=='two_moons']
# gauss_df = df[df["task"]=='gaussian_linear']
# glm_df = df[df["task"]=='bernoulli_glm']

names = ['two_moons', 'bernoulli_glm', 'gaussian_linear', 'slcp', 'lotka_volterra', 'sir']

In [9]:
means = []

for name in names:
    task_df = df[df["task"]==name]
    print(f"===== Task: {name}")

    ess = extract_diagnostic(task_df, "iw var")
    for ess_for_given_sim_budget in ess[:3]:
        x = np.asarray(ess_for_given_sim_budget)[:, 1:]
        minimal_ess = np.min(x)
        mean_ess = np.mean(x)
        print("minimal_ess", minimal_ess)
        print("mean_ess", mean_ess)
        
        means.append(mean_ess)

===== Task: two_moons
minimal_ess 6.49294424057006
mean_ess 26.335870928234524
minimal_ess 5.758741855621338
mean_ess 15.288498751322429
minimal_ess 5.873104095458984
mean_ess 13.41502628326416
===== Task: bernoulli_glm
minimal_ess 5.752330780029297
mean_ess 8.697088781992589
minimal_ess 5.512922763824463
mean_ess 8.311622953414917
minimal_ess 5.728072643280029
mean_ess 9.688377989663017
===== Task: gaussian_linear
minimal_ess 12.457694053649902
mean_ess 61.89263153076172
minimal_ess 6.561574935913086
mean_ess 15.598574935065365
minimal_ess 7.18271017074585
mean_ess 14.745691394805899
===== Task: slcp
minimal_ess 6.45048952102661
mean_ess 105.1868927637736
minimal_ess 5.71458625793457
mean_ess 32.9552890088823
minimal_ess 5.242841720581055
mean_ess 12.885489061143662
===== Task: lotka_volterra
minimal_ess 3.8183417320251465
mean_ess 48.30769086943732
minimal_ess 3.8124301433563232
mean_ess 7.8092647420035455
minimal_ess 2.5467476844787598
mean_ess 5.722040960523816
===== Task: sir
mini

In [10]:
print("Average ESS: ", np.mean(np.asarray(means)))

Average ESS:  25.154718082186616
